In [1]:
import sys, flyqma
sys.modules['clones'] = flyqma

from os.path import join
import numpy as np
import matplotlib.pyplot as plt
from flyqma.visualization.settings import *
%matplotlib inline

In [2]:
from flyqma.data import Experiment
experiment_path = '../../../data/published/real'
experiment = Experiment(experiment_path)

In [49]:
def rename_layer(layer):

    for i, ch in enumerate('bgr'):
        for base in ('', '_std', '_radial', '_zscore', '_community', '_normalized', '_predicted', 'c', 'c_normalized'):
            OLD_NAME = ch + base
            NEW_NAME = 'ch' + str(i) + base
            
            try:
                if OLD_NAME in layer.data.columns:
                    layer.data.rename({OLD_NAME: NEW_NAME}, axis='columns', inplace=True)
            except:
                pass
                
            try:
                if OLD_NAME in layer.measurements.columns:
                    layer.measurements.rename({OLD_NAME: NEW_NAME}, axis='columns', inplace=True)
            except:
                pass

        keydict = {v:'ch'+str(k) for k,v in dict(enumerate('bgr')).items()}
        
        try:
            if layer.metadata['bg'] in keydict.keys():
                layer.metadata['bg'] = keydict[layer.metadata['bg']]
        except:
            pass
        
        try:
            if 'graph_weighted_by' in layer.metadata['params'].keys():
                parts = layer.metadata['params']['graph_weighted_by'].split('_')
                if parts[0] in keydict.keys():        
                    parts[0] = keydict[parts[0]]
                    layer.metadata['params']['graph_weighted_by'] = '_'.join(parts)
        except:
            pass

        try:
            if layer.annotator.attribute in keydict.keys():
                layer.annotator.attribute = keydict[layer.annotator.attribute]
        except:
            pass
        
        try:
            for j, attr in enumerate(layer.annotator.classifier.attribute):

                parts = attr.split('_')
                if parts[0] in keydict.keys():
                    parts[0] = keydict[parts[0]]
                    layer.annotator.classifier.attribute[j] = '_'.join(parts)
                    layer.annotator.classifier.parameters['attribute'][j] = '_'.join(parts)
        except:
            pass

        
def convert(stack):
    for layer_id in range(stack.depth):
        layer = stack[layer_id]
        rename_layer(layer)
        if layer.include:
            layer.save(annotator=True)
    stack.save()
    stack.save_annotator()

In [52]:
for stack_id in experiment.stack_ids:
    stack = experiment.load_stack(stack_id, full=True)
    convert(stack)